In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((12510, 14), (12510, 14))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove columns that aren't relevant
df = df.drop(['fold', 'PL', 'vit', 'model', 'data', 'ALL',  'kw'], axis = 1) #'method', 'AL.iter', 'ratio',

# Replace 'None' with NaN, to allow conversion to numerical
df['AL.iter'] = df['AL.iter'].replace('None', np.nan)
df['AL.iter'] = pd.to_numeric(df['AL.iter'])
df['AL.epochs'] = df['AL.epochs'].replace('None', np.nan)
df['AL.epochs'] = pd.to_numeric(df['AL.epochs'])

cols = sorted(df.columns.tolist())
print(cols)
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value'])), dropna = False).agg({'value':['mean', 'std', 'count']})

['AL.epochs', 'AL.iter', 'bs', 'dataset', 'epochs', 'lr', 'method', 'metric', 'ratio', 'value']


,dataset,metric,value,ratio,method,AL.iter,AL.epochs,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,60.99,0.1,base,NaN,NaN,5,0.00005,64
1,UCM-CLS,zeroshot-val-top1,56.84,0.1,base,NaN,NaN,5,0.00005,64
2,RSICD,image_to_text_R@1,4.76,0.1,base,NaN,NaN,5,0.00005,64
3,RSICD,image_to_text_R@5,20.05,0.1,base,NaN,NaN,5,0.00005,64
4,RSICD,image_to_text_R@10,33.88,0.1,base,NaN,NaN,5,0.00005,64
...,...,...,...,...,...,...,...,...,...,...
12505,Sydney,image_to_text_R@5,24.75,0.7,base,3.0,15.0,15,0.00005,64
12506,Sydney,image_to_text_R@10,36.62,0.7,base,3.0,15.0,15,0.00005,64
12507,Sydney,text_to_image_R@1,7.65,0.7,base,3.0,15.0,15,0.00005,64
12508,Sydney,text_to_image_R@5,23.54,0.7,base,3.0,15.0,15,0.00005,64


In [6]:
display(df_grouped)

value  \
                                                                                mean   
AL.epochs AL.iter dataset ratio metric             epochs method lr      bs            
1.0       3.0     RSICD   0.1   image_to_text_R@1  1      base   0.00001 64   1.1000   
                                image_to_text_R@10 1      base   0.00001 64   6.3200   
                                image_to_text_R@5  1      base   0.00001 64   3.7500   
                                text_to_image_R@1  1      base   0.00001 64   1.0100   
                                text_to_image_R@10 1      base   0.00001 64   5.5900   
...                                                                              ...   
NaN       NaN     UCM-CLS 0.5   zeroshot-val-top1  10     base   0.00005 64  77.5375   
                                                   15     base   0.00005 64  78.2875   
                          0.7   zeroshot-val-top1  5      base   0.00005 64  78.1875   
                                                   10     base   0.00005 64  78.3975   
                                                   15     base   0.00005 64  77.9725   

                                                                                       \
                                                                                  std   
AL.epochs AL.iter dataset ratio metric             epochs method lr      bs             
1.0       3.0     RSICD   0.1   image_to_text_R@1  1      base   0.00001 64       NaN   
                                image_to_text_R@10 1      base   0.00001 64       NaN   
                                image_to_text_R@5  1      base   0.00001 64       NaN   
                                text_to_image_R@1  1      base   0.00001 64       NaN   
                                text_to_image_R@10 1      base   0.00001 64       NaN   
...                                                                               ...   
NaN       NaN     UCM-CLS 0.5   zeroshot-val-top1  10     base   0.00005 64  4.132662   
                                                   15     base   0.00005 64  3.939225   
                          0.7   zeroshot-val-top1  5      base   0.00005 64  5.256433   
                                                   10     base   0.00005 64  3.115032   
                                                   15     base   0.00005 64  3.209884   

                                                                                   
                                                                            count  
AL.epochs AL.iter dataset ratio metric             epochs method lr      bs        
1.0       3.0     RSICD   0.1   image_to_text_R@1  1      base   0.00001 64     1  
                                image_to_text_R@10 1      base   0.00001 64     1  
                                image_to_text_R@5  1      base   0.00001 64     1  
                                text_to_image_R@1  1      base   0.00001 64     1  
                                text_to_image_R@10 1      base   0.00001 64     1  
...                                                                           ...  
NaN       NaN     UCM-CLS 0.5   zeroshot-val-top1  10     base   0.00005 64     4  
                                                   15     base   0.00005 64     4  
                          0.7   zeroshot-val-top1  5      base   0.00005 64     4  
                                                   10     base   0.00005 64     4  
                                                   15     base   0.00005 64     4  

[1820 rows x 3 columns]

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_results = df_grouped.reset_index()
df_results['AL.epochs'] = df_results['AL.epochs'].fillna(df_results['epochs'])
df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])
# df_results = df_results.groupby(['metric', 'dataset'])
# df_results = df_grouped.reset_index()
# df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])

# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
# df_2 = df_2[(df_2['epochs'] > 20)  & (df_2['bs'] == 64) & (df_2['lr']==5e-5)] # &(df_2['lr']==5e-5)
# df_2 = df_2[(df_2['lr']==5e-5) & (df_2['bs'] == 64) & (df_2['method'] == 'base')] # &(df_2['lr']==5e-5)
display(df_results)

AL.epochs AL.iter  dataset ratio             metric epochs method  \
                                                                         
586        5.0     NaN    RSICD   0.1  image_to_text_R@1      5   ours   
587        5.0     NaN    RSICD   0.1  image_to_text_R@1      5   ours   
617       25.0     NaN    RSICD   0.1  image_to_text_R@1     25   ours   
624       30.0     NaN    RSICD   0.1  image_to_text_R@1     30   ours   
594       10.0     NaN    RSICD   0.1  image_to_text_R@1     10   ours   
...        ...     ...      ...   ...                ...    ...    ...   
1814       5.0     NaN  UCM-CLS   0.5  zeroshot-val-top1      5   base   
1819      15.0     NaN  UCM-CLS   0.7  zeroshot-val-top1     15   base   
1817       5.0     NaN  UCM-CLS   0.7  zeroshot-val-top1      5   base   
1816      15.0     NaN  UCM-CLS   0.5  zeroshot-val-top1     15   base   
1818      10.0     NaN  UCM-CLS   0.7  zeroshot-val-top1     10   base   

           lr   bs    value                  
                       mean       std count  
586   0.00050   64   0.0700  0.060000     9  
587   0.00050  128   0.0800  0.030000     9  
617   0.00050   64   0.0800  0.030000     9  
624   0.00050   64   0.0900  0.045000     9  
594   0.00050   64   0.1000  0.030000     9  
...       ...  ...      ...       ...   ...  
1814  0.00005   64  77.7550  3.436680     4  
1819  0.00005   64  77.9725  3.209884     4  
1817  0.00005   64  78.1875  5.256433     4  
1816  0.00005   64  78.2875  3.939225     4  
1818  0.00005   64  78.3975  3.115032     4  

[1820 rows x 12 columns]

In [9]:
df_results.shape

(1820, 12)

In [13]:
# Results with active learning
full_display(df_results[(df_results['AL.iter']>0) & (df_results['epochs']==15)]) #.groupby(['metric', 'dataset']).tail(3)

AL.epochs AL.iter    dataset ratio              metric epochs method  \
                                                                           
220      15.0     3.0      RSICD   0.1   image_to_text_R@1     15   base   
302      15.0     1.0      RSICD   0.1   image_to_text_R@1     15   base   
441      15.0     5.0      RSICD   0.1   image_to_text_R@1     15   base   
381      15.0     3.0      RSICD   0.1   image_to_text_R@1     15   base   
226      15.0     3.0      RSICD   0.3   image_to_text_R@1     15   base   
232      15.0     3.0      RSICD   0.5   image_to_text_R@1     15   base   
238      15.0     3.0      RSICD   0.7   image_to_text_R@1     15   base   
402      15.0     3.0     Sydney   0.1   image_to_text_R@1     15   base   
248      15.0     3.0     Sydney   0.1   image_to_text_R@1     15   base   
462      15.0     5.0     Sydney   0.1   image_to_text_R@1     15   base   
330      15.0     1.0     Sydney   0.1   image_to_text_R@1     15   base   
254      15.0     3.0     Sydney   0.3   image_to_text_R@1     15   base   
260      15.0     3.0     Sydney   0.5   image_to_text_R@1     15   base   
266      15.0     3.0     Sydney   0.7   image_to_text_R@1     15   base   
354      15.0     1.0        UCM   0.1   image_to_text_R@1     15   base   
420      15.0     3.0        UCM   0.1   image_to_text_R@1     15   base   
480      15.0     5.0        UCM   0.1   image_to_text_R@1     15   base   
272      15.0     3.0        UCM   0.1   image_to_text_R@1     15   base   
284      15.0     3.0        UCM   0.5   image_to_text_R@1     15   base   
278      15.0     3.0        UCM   0.3   image_to_text_R@1     15   base   
290      15.0     3.0        UCM   0.7   image_to_text_R@1     15   base   
306      15.0     1.0      RSICD   0.1  image_to_text_R@10     15   base   
221      15.0     3.0      RSICD   0.1  image_to_text_R@10     15   base   
384      15.0     3.0      RSICD   0.1  image_to_text_R@10     15   base   
444      15.0     5.0      RSICD   0.1  image_to_text_R@10     15   base   
227      15.0     3.0      RSICD   0.3  image_to_text_R@10     15   base   
233      15.0     3.0      RSICD   0.5  image_to_text_R@10     15   base   
239      15.0     3.0      RSICD   0.7  image_to_text_R@10     15   base   
405      15.0     3.0     Sydney   0.1  image_to_text_R@10     15   base   
249      15.0     3.0     Sydney   0.1  image_to_text_R@10     15   base   
465      15.0     5.0     Sydney   0.1  image_to_text_R@10     15   base   
334      15.0     1.0     Sydney   0.1  image_to_text_R@10     15   base   
255      15.0     3.0     Sydney   0.3  image_to_text_R@10     15   base   
261      15.0     3.0     Sydney   0.5  image_to_text_R@10     15   base   
267      15.0     3.0     Sydney   0.7  image_to_text_R@10     15   base   
358      15.0     1.0        UCM   0.1  image_to_text_R@10     15   base   
423      15.0     3.0        UCM   0.1  image_to_text_R@10     15   base   
273      15.0     3.0        UCM   0.1  image_to_text_R@10     15   base   
483      15.0     5.0        UCM   0.1  image_to_text_R@10     15   base   
279      15.0     3.0        UCM   0.3  image_to_text_R@10     15   base   
285      15.0     3.0        UCM   0.5  image_to_text_R@10     15   base   
291      15.0     3.0        UCM   0.7  image_to_text_R@10     15   base   
310      15.0     1.0      RSICD   0.1   image_to_text_R@5     15   base   
222      15.0     3.0      RSICD   0.1   image_to_text_R@5     15   base   
447      15.0     5.0      RSICD   0.1   image_to_text_R@5     15   base   
387      15.0     3.0      RSICD   0.1   image_to_text_R@5     15   base   
228      15.0     3.0      RSICD   0.3   image_to_text_R@5     15   base   
234      15.0     3.0      RSICD   0.5   image_to_text_R@5     15   base   
240      15.0     3.0      RSICD   0.7   image_to_text_R@5     15   base   
408      15.0     3.0     Sydney   0.1   image_to_text_R@5     15   base   
250      15.0     3.0     Sydney   0.1   image_to_te

In [11]:
# Results for base CLIP model (filtered a bit)
full_display(df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'base') & (df_results['lr']==5e-5)].groupby(['metric', 'dataset']).tail(3))

AL.epochs AL.iter    dataset ratio              metric epochs method  \
                                                                            
918       15.0     NaN      RSICD   0.5   image_to_text_R@1     15   base   
935       10.0     NaN      RSICD   0.7   image_to_text_R@1     10   base   
936       15.0     NaN      RSICD   0.7   image_to_text_R@1     15   base   
1352      15.0     NaN     Sydney   0.5   image_to_text_R@1     15   base   
1369      10.0     NaN     Sydney   0.7   image_to_text_R@1     10   base   
1370      15.0     NaN     Sydney   0.7   image_to_text_R@1     15   base   
1724      15.0     NaN        UCM   0.5   image_to_text_R@1     15   base   
1741      10.0     NaN        UCM   0.7   image_to_text_R@1     10   base   
1742      15.0     NaN        UCM   0.7   image_to_text_R@1     15   base   
921       15.0     NaN      RSICD   0.5  image_to_text_R@10     15   base   
938       10.0     NaN      RSICD   0.7  image_to_text_R@10     10   base   
939       15.0     NaN      RSICD   0.7  image_to_text_R@10     15   base   
1372      10.0     NaN     Sydney   0.7  image_to_text_R@10     10   base   
1355      15.0     NaN     Sydney   0.5  image_to_text_R@10     15   base   
1373      15.0     NaN     Sydney   0.7  image_to_text_R@10     15   base   
1727      15.0     NaN        UCM   0.5  image_to_text_R@10     15   base   
1744      10.0     NaN        UCM   0.7  image_to_text_R@10     10   base   
1745      15.0     NaN        UCM   0.7  image_to_text_R@10     15   base   
924       15.0     NaN      RSICD   0.5   image_to_text_R@5     15   base   
941       10.0     NaN      RSICD   0.7   image_to_text_R@5     10   base   
942       15.0     NaN      RSICD   0.7   image_to_text_R@5     15   base   
1375      10.0     NaN     Sydney   0.7   image_to_text_R@5     10   base   
1358      15.0     NaN     Sydney   0.5   image_to_text_R@5     15   base   
1376      15.0     NaN     Sydney   0.7   image_to_text_R@5     15   base   
1730      15.0     NaN        UCM   0.5   image_to_text_R@5     15   base   
1747      10.0     NaN        UCM   0.7   image_to_text_R@5     10   base   
1748      15.0     NaN        UCM   0.7   image_to_text_R@5     15   base   
927       15.0     NaN      RSICD   0.5   text_to_image_R@1     15   base   
944       10.0     NaN      RSICD   0.7   text_to_image_R@1     10   base   
945       15.0     NaN      RSICD   0.7   text_to_image_R@1     15   base   
1378      10.0     NaN     Sydney   0.7   text_to_image_R@1     10   base   
1361      15.0     NaN     Sydney   0.5   text_to_image_R@1     15   base   
1379      15.0     NaN     Sydney   0.7   text_to_image_R@1     15   base   
1733      15.0     NaN        UCM   0.5   text_to_image_R@1     15   base   
1750      10.0     NaN        UCM   0.7   text_to_image_R@1     10   base   
1751      15.0     NaN        UCM   0.7   text_to_image_R@1     15   base   
930       15.0     NaN      RSICD   0.5  text_to_image_R@10     15   base   
947       10.0     NaN      RSICD   0.7  text_to_image_R@10     10   base   
948       15.0     NaN      RSICD   0.7  text_to_image_R@10     15   base   
1381      10.0     NaN     Sydney   0.7  text_to_image_R@10     10   base   
1364      15.0     NaN     Sydney   0.5  text_to_image_R@10     15   base   
1382      15.0     NaN     Sydney   0.7  text_to_image_R@10     15   base   
1736      15.0     NaN        UCM   0.5  text_to_image_R@10     15   base   
1753      10.0     NaN        UCM   0.7  text_to_image_R@10     10   base   
1754      15.0     NaN        UCM   0.7  text_to_image_R@10     15   base   
933       15.0     NaN      RSICD   0.5   text_to_image_R@5     15   base   
950       10.0     NaN      RSICD   0.7   text_to_image_R@5     10   base   
951       15.0     NaN      RSICD   0.7   text_to_image_R@5     15   base   
1384      10.0     NaN     Sydney   0.7   text_to_image_R@5     10   base   
1367      15.0     NaN     Sydney   0.5   text_to_image_R@5     15   base   
1

In [12]:
# Results for S-CLIP model (filtered a bit)
full_display(df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['lr']==5e-5)].groupby(['metric', 'dataset']).tail(3))

AL.epochs AL.iter    dataset ratio              metric epochs method  \
                                                                            
616       25.0     NaN      RSICD   0.1   image_to_text_R@1     25   ours   
623       30.0     NaN      RSICD   0.1   image_to_text_R@1     30   ours   
630       35.0     NaN      RSICD   0.1   image_to_text_R@1     35   ours   
1035      15.0     NaN     Sydney   0.1   image_to_text_R@1     15   ours   
1057      30.0     NaN     Sydney   0.1   image_to_text_R@1     30   ours   
1064      35.0     NaN     Sydney   0.1   image_to_text_R@1     35   ours   
1422      25.0     NaN        UCM   0.1   image_to_text_R@1     25   ours   
1429      30.0     NaN        UCM   0.1   image_to_text_R@1     30   ours   
1436      35.0     NaN        UCM   0.1   image_to_text_R@1     35   ours   
669       25.0     NaN      RSICD   0.1  image_to_text_R@10     25   ours   
683       35.0     NaN      RSICD   0.1  image_to_text_R@10     35   ours   
676       30.0     NaN      RSICD   0.1  image_to_text_R@10     30   ours   
1103      25.0     NaN     Sydney   0.1  image_to_text_R@10     25   ours   
1110      30.0     NaN     Sydney   0.1  image_to_text_R@10     30   ours   
1117      35.0     NaN     Sydney   0.1  image_to_text_R@10     35   ours   
1488      35.0     NaN        UCM   0.1  image_to_text_R@10     35   ours   
1482      30.0     NaN        UCM   0.1  image_to_text_R@10     30   ours   
1489      35.0     NaN        UCM   0.1  image_to_text_R@10     35   ours   
722       25.0     NaN      RSICD   0.1   image_to_text_R@5     25   ours   
729       30.0     NaN      RSICD   0.1   image_to_text_R@5     30   ours   
736       35.0     NaN      RSICD   0.1   image_to_text_R@5     35   ours   
1162      30.0     NaN     Sydney   0.1   image_to_text_R@5     30   ours   
1163      30.0     NaN     Sydney   0.1   image_to_text_R@5     30   ours   
1170      35.0     NaN     Sydney   0.1   image_to_text_R@5     35   ours   
1542      35.0     NaN        UCM   0.1   image_to_text_R@5     35   ours   
1535      30.0     NaN        UCM   0.1   image_to_text_R@5     30   ours   
1528      25.0     NaN        UCM   0.1   image_to_text_R@5     25   ours   
782       30.0     NaN      RSICD   0.1   text_to_image_R@1     30   ours   
789       35.0     NaN      RSICD   0.1   text_to_image_R@1     35   ours   
788       35.0     NaN      RSICD   0.1   text_to_image_R@1     35   ours   
1223      35.0     NaN     Sydney   0.1   text_to_image_R@1     35   ours   
1216      30.0     NaN     Sydney   0.1   text_to_image_R@1     30   ours   
1209      25.0     NaN     Sydney   0.1   text_to_image_R@1     25   ours   
1581      25.0     NaN        UCM   0.1   text_to_image_R@1     25   ours   
1588      30.0     NaN        UCM   0.1   text_to_image_R@1     30   ours   
1595      35.0     NaN        UCM   0.1   text_to_image_R@1     35   ours   
828       25.0     NaN      RSICD   0.1  text_to_image_R@10     25   ours   
835       30.0     NaN      RSICD   0.1  text_to_image_R@10     30   ours   
842       35.0     NaN      RSICD   0.1  text_to_image_R@10     35   ours   
1262      25.0     NaN     Sydney   0.1  text_to_image_R@10     25   ours   
1269      30.0     NaN     Sydney   0.1  text_to_image_R@10     30   ours   
1276      35.0     NaN     Sydney   0.1  text_to_image_R@10     35   ours   
1641      30.0     NaN        UCM   0.1  text_to_image_R@10     30   ours   
1647      35.0     NaN        UCM   0.1  text_to_image_R@10     35   ours   
1648      35.0     NaN        UCM   0.1  text_to_image_R@10     35   ours   
881       25.0     NaN      RSICD   0.1   text_to_image_R@5     25   ours   
895       35.0     NaN      RSICD   0.1   text_to_image_R@5     35   ours   
888       30.0     NaN      RSICD   0.1   text_to_image_R@5     30   ours   
1315      25.0     NaN     Sydney   0.1   text_to_image_R@5     25   ours   
1322      30.0     NaN     Sydney   0.1   text_to_image_R@5     30   ours   
1